In [1]:
import os
import json
from transformers import pipeline
from tqdm import tqdm

## Add new metric over existing completions

We don't want to recompute the expensive part of the grid, so we just
add metrics on top of existing completions.

In [2]:
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english", device="cuda")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
sentiment_analysis(["I love you", "I hate you"])

[{'label': 'POSITIVE', 'score': 0.998561680316925},
 {'label': 'NEGATIVE', 'score': 0.9991401433944702}]

In [4]:
def add_sentiment(exp_name):
    for f in tqdm(os.listdir(f"grid_results/{exp_name}"), desc=f"adding sentiment to {exp_name}"):
        if f == "meta.json":
            continue

        with open(f"grid_results/{exp_name}/{f}") as fp:
            d = json.load(fp)

        if "sentiment" in d:
            continue # already computed

        generations = [c[len(p):].replace("<|end_of_text|>", "") for c, p in zip(d["completions"], d["prompt_batch"])]
        results = sentiment_analysis(generations)
        prob_positive = [r["score"] if r["label"] == "POSITIVE" else 1 - r["score"] for r in results]
        d["sentiment"] = prob_positive

        with open(f"grid_results/{exp_name}/{f}", "w") as fp:
            json.dump(d, fp)

for exp_name in os.listdir("grid_results"):
    add_sentiment(exp_name)

adding sentiment to positive_n1024: 100%|██████████| 1025/1025 [00:00<00:00, 7440.97it/s]


In [5]:
import time
while True:
    time.sleep(60)
    for exp_name in os.listdir("grid_results"):
        add_sentiment(exp_name)

KeyboardInterrupt: 

## Recompute perplexity


In [2]:
import torch
from transformer_lens import HookedTransformer
import numpy as np
import pandas as pd

In [7]:

torch.set_grad_enabled(False)  # save memory
model_name = "meta-llama/Meta-Llama-3-8B"
# model_name = "gpt2-xl"
device = "cuda:0"
model = HookedTransformer.from_pretrained(model_name, device=device)
model.eval()
if torch.cuda.is_available():
  model.to(device)

model.device = device

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded pretrained model meta-llama/Meta-Llama-3-8B into HookedTransformer
Moving model to device:  cuda:0


In [3]:
# same function but do the forward pass for every input at once
def compute_perplexities_batched(dataset, model, tokenizer, batch_size=8, **kwargs):
    perplexities = []
    sum_nllos = 0
    n_tokens = 0
    
    for i in range(0, len(dataset), batch_size):
        batch = dataset.iloc[i:i+batch_size]
        
        # Get input lengths for each sample in batch
        input_lengths = []
        combined_sentences = []
        for _, sample in batch.iterrows():
            if len(sample['generated']) == 0:
                input_lengths.append(0)
                continue
            input_encodings = tokenizer(sample['input'], return_tensors='pt')
            input_lengths.append(input_encodings['input_ids'].size(1))
            combined_sentences.append(sample['input'] + sample['generated'])
            
        if not combined_sentences:
            continue
            
        # Tokenize full batch
        encodings = tokenizer(combined_sentences, return_tensors='pt', padding=True)
        input_ids = encodings['input_ids'].to(model.device)
        attention_mask = encodings['attention_mask'].to(model.device)
        
        import time
        start = time.time()
        with torch.no_grad():
            # logits = model(input_ids, labels=input_ids, attention_mask=attention_mask).logits # hf_model
            logits = model(input_ids, attention_mask=attention_mask) # transformerlens model
            logprobs = logits.log_softmax(dim=-1)
        end = time.time()
        print(f"time: {end - start}")
        
        loss_func = torch.nn.NLLLoss(ignore_index=-100, reduction='sum')
        
        # Calculate loss for each sample in batch
        for j in range(len(combined_sentences)):
            input_length = input_lengths[j]
            if input_length == 0:
                perplexities.append(None)
                continue
                
            loss = loss_func(
                logprobs[j, input_length:-1, :].contiguous(),
                input_ids[j, input_length+1:].contiguous()
            )
            loss = loss.to(torch.float32).detach().cpu().numpy()
            
            n_tokens_here = (attention_mask[j, input_length+1:] == 1).sum().item()
            
            if n_tokens_here > 0:
                perplexity = np.exp(loss / n_tokens_here)
                sum_nllos += loss
                n_tokens += n_tokens_here
                if not np.isnan(perplexity):
                    perplexities.append(perplexity)
                else:
                    perplexities.append(None)
            else:
                perplexities.append(None)
                
    return perplexities, sum_nllos, n_tokens


def get_perplexity(dataset, model, **kwargs):
    """
    Calculates the perplexity of the generated sentences.

    Args:
        dataset (pd.DataFrame): The dataset to be used for evaluation. Has columns "input" (for input text), "generated" (for generated text). 
        model (PreTrainedModel): The model to be evaluated.
        tokenizer (Tokenizer): The tokenizer to be used for tokenizing the sentences.
        **kwargs: Additional keyword arguments.
    """
    perplexities, sum_nllos, n_tokens = compute_perplexities_batched(dataset, model, model.tokenizer, **kwargs)

    return {
        "average": np.mean(perplexities),
        "median": np.median(perplexities),
        "correct_perplexity": np.exp(sum_nllos / n_tokens)
    }

In [ ]:
# TODO: Check batched fn. with different prompts (ugh)

In [19]:
def recompute_perplexity_and_sentiment(exp_name):
    for f in tqdm(os.listdir(f"grid_results/{exp_name}"), desc=f"adding sentiment to {exp_name}"):
        if f == "meta.json":
            continue

        with open(f"grid_results/{exp_name}/{f}") as fp:
            d = json.load(fp)

        # recompute perplexity
        generated_only = [c[len(p):].replace("<|end_of_text|>", "") for c, p in zip(d["completions"], d["prompt_batch"])]
        prompt_only = [p.replace("<|end_of_text|>", "") for p in d["prompt_batch"]]

        df = pd.DataFrame({"input": prompt_only, "generated": generated_only})
        perplexities, sum_nllos, n_tokens = compute_perplexities_batched(df, model, model.tokenizer, batch_size=100)
        d["correct_perplexity"] = np.exp(sum_nllos / n_tokens)
        d["perplexities_new"] = perplexities

        print(f"existing {np.mean(d['perplexities'])} -> {d['correct_perplexity']}")

        break

        # recompute sentiment
        completions = [c.replace("<|end_of_text|>", "") for c in d["completions"]]
        results = sentiment_analysis(completions)
        prob_positive = [r["score"] if r["label"] == "POSITIVE" else 1 - r["score"] for r in results]
        d["sentiment_new"] = prob_positive

        with open(f"grid_results/{exp_name}/{f}", "w") as fp:
            json.dump(d, fp)


for exp_name in os.listdir("grid_results"):
    recompute_perplexity_and_sentiment(exp_name)
    break

adding sentiment to negative_n1024_fp:   0%|          | 0/1025 [00:00<?, ?it/s]

time: 6.938963174819946


adding sentiment to negative_n1024_fp:   0%|          | 0/1025 [00:11<?, ?it/s]

existing 74.9001877784729 -> 127.94375292877415


In [4]:
exp_name = "positive_n64"

with open(f"grid_results/{exp_name}/9_l3_c-1.json") as fp:
    d = json.load(fp)

generated_only = [c[len(p):].replace("<|end_of_text|>", "") for c, p in zip(d["completions"], d["prompt_batch"])]
prompt_only = [p.replace("<|end_of_text|>", "") for p in d["prompt_batch"]]

df = pd.DataFrame({"input": prompt_only, "generated": generated_only})
df

,input,generated
0,Jean Dujardin gets Connery's mannerisms down p...,"the silly grin he flashes at the camera, even..."
1,I have seen so many bad reviews on Supervivien...,"First, I deal with the question of what would..."
2,"Looking backwards to that year 2002 when ""Furi...",". Comedians like Alina Pușcău, a veteran of Ro..."
3,THE CELL (2000) Rating: 8/10<br /><br />The Ce...,Once in 2D and once in 3D which is included o...
4,I loved this movie! It was all I could do not ...,by the great work done by the various actors ...
5,- After their sons are sentenced to life in pr...,"calls from Brooks Halshaw (John Forsythe), th..."
6,I love playing football and I thought this mov...,"enjoyed watching this movie. However, there w..."
7,This Showtime cable film features a talented c...,experience to the sex addict who engages in r...
8,This is a film i decided to go and see because...,around kids in the same way that most animati...
9,This movie is my all time favorite!!! You real...,is definitely my Top 3 Michael Jackson Music ...


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

hf_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")
hf_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

hf_model.to("cuda:0")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm)

In [8]:
hf_model = hf_model.eval()

In [17]:
encodings = hf_tokenizer(d["completions"], return_tensors='pt')
input_ids = encodings['input_ids'].to(hf_model.device)
attention_mask = encodings['attention_mask'].to(hf_model.device)

In [19]:
hf_model.compile()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [29]:
import time

with torch.no_grad():
    start = time.time()
    logits = hf_model(input_ids, attention_mask=attention_mask)
    end = time.time()
    print(f"time: {end - start}")

time: 8.818042039871216
